# Important Graphs for analysis in the case of City

In [2]:
# -------------------------------
# Cell 1: Environment Setup
# -------------------------------
import os
import sys
import logging
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display, clear_output

# Import the custom analysis script
import solar_analysis_script as sas
from solar_analysis_script import update_weekly_graph  # The update function

# (Optional) Reload the module to pick up any changes during development
import importlib
importlib.reload(sas)

# -------------------------------
# Cell 2: Configuration and File Paths
# -------------------------------
# File paths for the data files
CONSUMPTION_FILE = 'Grenoble_city_load_1.csv'
PV_FILE = 'One_PV_Panel_Production.csv'
WIND_FILE = 'wind_production_for_one_turbine.csv'

# Date range for the year 2019
CONSUMPTION_START_DATE = '2019-01-01 00:00:00'
CONSUMPTION_END_DATE   = '2019-12-31 23:59:59'

# Initial parameter values (from the analysis script)
INITIAL_NUM_PANELS   = sas.INITIAL_NUM_PANELS
INITIAL_NUM_TURBINES = sas.INITIAL_NUM_TURBINES
INITIAL_NUM_BATTERIES = 500  # Adjust if needed
battery_capacity     = sas.battery_capacity_per_unit  # kWh per battery

# -------------------------------
# Cell 3: Load Raw Data
# -------------------------------
print("Loading and preprocessing consumption data...")
try:
    # Call the module-level function (which converts from MWh to kWh)
    # Note: Although the CSV column is in MWh, the function converts it to kWh.
    city_consumption = sas.load_city_consumption(
        consumption_file_path=CONSUMPTION_FILE,
        consumption_sep=';',
        consumption_date_col='Date',
        consumption_time_col='Heures',
        consumption_value_col='Consommation(MWh)'
    )
    print("Consumption Data (head):")
    print(city_consumption.head())
except Exception as err:
    print(f"Failed to load consumption data: {err}")
    raise

print("\nLoading and preprocessing PV production data...")
try:
    city_pv_production = sas.load_city_pv_production(
        pv_file_path=PV_FILE,
        consumption_start_date=CONSUMPTION_START_DATE,
        consumption_end_date=CONSUMPTION_END_DATE,
        sep=',',
        time_start_col='time',
        time_end_col='local_time',
        electricity_col='electricity'
    )
    print("PV Production Data (head):")
    print(city_pv_production.head())
except Exception as err:
    print(f"Failed to load PV production data: {err}")
    raise

print("\nLoading and preprocessing Wind production data...")
try:
    city_wind_production = sas.load_city_wind_production(
        wind_file_path=WIND_FILE,
        consumption_start_date=CONSUMPTION_START_DATE,
        consumption_end_date=CONSUMPTION_END_DATE,
        sep=',',
        time_start_col='time',
        time_end_col='local_time',
        electricity_col='wind_electricity'
    )
    print("Wind Production Data (head):")
    print(city_wind_production.head())
except Exception as err:
    print(f"Failed to load wind production data: {err}")
    raise

Loading and preprocessing consumption data...
Consumption Data (head):
Datetime
2019-01-01 00:00:00    580000
2019-01-01 01:00:00    558000
2019-01-01 02:00:00    539000
2019-01-01 03:00:00    543000
2019-01-01 04:00:00    531000
Name: Consumption_kWh, dtype: int64

Loading and preprocessing PV production data...
PV Production Data (head):
Time_Start
2019-01-01 00:00:00    0.0
2019-01-01 01:00:00    0.0
2019-01-01 02:00:00    0.0
2019-01-01 03:00:00    0.0
2019-01-01 04:00:00    0.0
Name: PV_Production_kWh, dtype: float64

Loading and preprocessing Wind production data...
Wind Production Data (head):
Time_Start
2019-01-01 00:00:00     923.138
2019-01-01 01:00:00     942.556
2019-01-01 02:00:00    1064.198
2019-01-01 03:00:00    1328.341
2019-01-01 04:00:00    1625.839
Name: Wind_Production_kWh, dtype: float64


In [2]:
# ---------------------------
# City-Level Annual SSR Analysis (PV Penetration Ratio)
# ---------------------------
# System parameters (ensure these match your case):
# Calculate the total PV production from city_pv_production (in kWh)
total_pv_power_production = city_pv_production.sum()

# Compute average production over the year (using 8760 hours/year)
average_pv_power_production = total_pv_power_production / 8760

# Calculate the total annual energy demand from city_consumption (in kWh)
total_energy_demand = city_consumption.sum()

# Compute average demand (kW) over the year (using 8760 hours/year)
average_demand = total_energy_demand / 8760

# Define the range of number of panels (absolute number) to analyze
num_panels_range = np.arange(0, 1e9, 5e6)  

# Initialize lists to store SSR values and PV penetration ratio
ssr_values = []
pv_penetration_ratio = []

# Loop over different numbers of panels
for num_panels in num_panels_range:
    installed_capacity = num_panels * average_pv_power_production
    ratio = installed_capacity / average_demand
    pv_penetration_ratio.append(ratio)

    total_energy_supplied_pv = 0.0
    
    for timestamp in city_consumption.index:
        consumption = city_consumption.loc[timestamp]  # kWh
        pv_production = city_pv_production.loc[timestamp] * num_panels * average_pv_power_production
        energy_supplied = min(pv_production, consumption)
        total_energy_supplied_pv += energy_supplied

    ssr = (total_energy_supplied_pv / total_energy_demand) * 100 if total_energy_demand > 0 else 0
    ssr_values.append(ssr)

# Plot SSR vs. PV Penetration Ratio with hover information
fig_ssr = go.Figure()

fig_ssr.add_trace(go.Scatter(
    x=pv_penetration_ratio,
    y=ssr_values,
    mode='lines+markers',
    name='SSR (%)',
    line=dict(color='green'),
    marker=dict(size=8),
    hovertemplate='<b>PV Penetration Ratio:</b> %{x:.4f}<br>'
                  '<b>SSR (%):</b> %{y:.2f}<br>'
                  '<b>Number of Panels:</b> %{customdata:,}',
    customdata=num_panels_range  # Pass number of panels as custom data for hover tooltips
))

fig_ssr.update_layout(
    title='City-Level Self-Sufficiency Rate (SSR) vs. PV Penetration Ratio',
    xaxis_title='PV Penetration Ratio (Installed PV Capacity / Average Demand)',
    yaxis_title='Self-Sufficiency Rate (Total Energy Supplied / Total Energy demand)',
    template='plotly_white',
    width=800,
    height=600,
    font=dict(size=12)
)

fig_ssr.show()


In [5]:
import numpy as np
import plotly.graph_objects as go

# ------------------------------------------------
# 1) Basic Parameters
# ------------------------------------------------
# city_consumption : pandas Series with your city’s hourly consumption in kWh
# city_wind_production : pandas Series with your wind turbines’ hourly production in kWh 
#   (or for a single “reference turbine,” in which case we multiply by # of turbines)

# -- total annual wind production (kWh) --
total_wind_production = city_wind_production.sum()

# -- average annual wind production (kWh/h) (assuming 8760 hours/year) --
average_wind_production = total_wind_production / 8760

# -- total annual demand (kWh) --
total_energy_demand = city_consumption.sum()

# -- average demand (kW) => total kWh / 8760 hours
average_demand = total_energy_demand / 8760

# ------------------------------------------------
# 2) Range of turbine counts
# ------------------------------------------------
# Example: from 0 to 1e6 in steps of 1e2  (adjust as you see fit)
num_turbines_range = np.arange(0, 1e3, 1e2)

# Prepare arrays to store results
turbine_penetration_ratios = []
ssr_values = []

# ------------------------------------------------
# 3) Loop Over Number of Turbines
# ------------------------------------------------
for num_turbines in num_turbines_range:
    # (A) Compute installed wind capacity
    #     Here we multiply “num_turbines * average_wind_production”
    #     to get total kW, since average_wind_production is kWh/h for 1 “reference turbine.”
    #     Adjust if your data is in different units.
    installed_capacity_kW = num_turbines * average_wind_production  # [kW]
    
    # (B) Penetration Ratio
    if average_demand > 0:
        ratio = installed_capacity_kW / average_demand
    else:
        ratio = 0
    turbine_penetration_ratios.append(ratio)
    
    # (C) Calculate how much wind meets the load hour by hour
    total_energy_supplied_by_wind = 0.0
    
    for timestamp in city_consumption.index:
        consumption_kWh = city_consumption.loc[timestamp]
        # hourly wind production for the chosen # of turbines
        wind_generation_kWh = city_wind_production.loc[timestamp] * num_turbines
        
        # energy that directly meets the consumption
        energy_supplied = min(wind_generation_kWh, consumption_kWh)
        total_energy_supplied_by_wind += energy_supplied
    
    # (D) SSR (in %)
    if total_energy_demand > 0:
        ssr = (total_energy_supplied_by_wind / total_energy_demand) * 100.0
    else:
        ssr = 0.0
    
    ssr_values.append(ssr)

# ------------------------------------------------
# 4) Plot: SSR vs. Wind Penetration Ratio
# ------------------------------------------------
fig_wind = go.Figure()

fig_wind.add_trace(go.Scatter(
    x=turbine_penetration_ratios,
    y=ssr_values,
    mode='lines+markers',
    name='SSR (%)',
    line=dict(color='blue'),
    marker=dict(size=8),
    hovertemplate=(
        '<b>Wind Penetration Ratio</b>: %{x:.4f}<br>'
        '<b>SSR (%):</b> %{y:.2f}<br>'
        '<b># of Turbines:</b> %{customdata:,}'
    ),
    customdata=num_turbines_range  # So we can display # of turbines in the hover
))

fig_wind.update_layout(
    title='City-Level SSR vs. Wind Turbine Penetration Ratio',
    xaxis_title='Wind Penetration Ratio (Installed Wind Capacity / Average Demand)',
    yaxis_title='Self-Sufficiency Rate (%)',
    template='plotly_white',
    width=800,
    height=600,
    font=dict(size=12)
)

fig_wind.show()


In [3]:
import plotly.graph_objects as go
import numpy as np

# ---------------------------------------------------------------------------
# City-Level Annual Analysis: SSR vs. PV & Wind Penetration Ratios
# ---------------------------------------------------------------------------
# This analysis uses production data for:
#  - One PV panel production (from 'One_PV_Panel_Production.csv')
#  - One wind turbine production (from 'wind_production_for_one_turbine.csv')

# Calculate the total PV production from city_pv_production (in kWh)
total_pv_power_production = city_pv_production.sum()

# Compute average production over the year (using 8760 hours/year)
average_pv_power_production = total_pv_power_production / 8760

# Calculate the total Wind production from city_wind_production (in kWh)
total_wind_power_production = city_wind_production.sum()

# Compute average production over the year (using 8760 hours/year)
average_wind_power_production = total_wind_power_production / 8760

# (No battery storage is modeled in this analysis.)
#
# Calculate the total annual energy demand (kWh) and average demand (kW)
total_energy_demand = city_consumption.sum()  # kWh per year
average_demand = total_energy_demand / 8760   # kW average demand (8760 hours/year)

# Define the range for the number of PV panels and wind turbines (raw counts)
raw_panel_range = np.arange(0, 5e8, 5e6)   
turbine_range = np.arange(0, 50, 10)           

# Create meshgrids for raw PV panel count and wind turbine count
panel_grid, turbine_grid = np.meshgrid(raw_panel_range, turbine_range)

# Compute the normalized penetration ratios:
# PV Penetration Ratio = (num_panels * panel_capacity) / average_demand
pv_penetration_grid = (panel_grid * average_pv_power_production) / average_demand
# Wind Penetration Ratio = (num_turbines * wind_turbine_capacity) / average_demand
wind_penetration_grid = (turbine_grid * average_wind_power_production) / average_demand

# Initialize a grid to store the SSR values (in percent)
ssr_grid = np.zeros_like(panel_grid, dtype=float)

# Loop over each combination of panel and turbine counts
for i in range(panel_grid.shape[0]):
    for j in range(panel_grid.shape[1]):
        num_panels = panel_grid[i, j]
        num_turbines = turbine_grid[i, j]
        
        total_energy_supplied = 0.0  # accumulator in kWh
        
        # Loop through each hourly timestamp
        for timestamp in city_consumption.index:
            consumption = city_consumption.loc[timestamp]  # kWh at this hour
            
            # PV production: scale one-panel production by num_panels and panel_capacity
            pv_production = city_pv_production.loc[timestamp] * num_panels * average_pv_power_production
            # Wind production: scale one-turbine production by num_turbines and wind_turbine_capacity
            wind_production = city_wind_production.loc[timestamp] * num_turbines * average_wind_power_production
            
            # Total production available (kWh)
            total_production = pv_production + wind_production
            
            # The energy supplied to meet consumption is the minimum of production and consumption
            energy_supplied = min(total_production, consumption)
            total_energy_supplied += energy_supplied
        
        # Compute the Self-Sufficiency Rate (SSR) as a percentage of annual demand met
        ssr = (total_energy_supplied / total_energy_demand) * 100 if total_energy_demand > 0 else 0
        ssr_grid[i, j] = ssr

# Create a 3D Surface Plot:
#   x-axis: PV Penetration Ratio,
#   y-axis: Wind Penetration Ratio,
#   z-axis: SSR (%)
fig = go.Figure(data=[go.Surface(
    x=pv_penetration_grid,
    y=wind_penetration_grid,
    z=ssr_grid,
    colorscale='Viridis',
    colorbar=dict(title="SSR (%)")
)])

fig.update_layout(
    title='City-Level Self-Sufficiency Rate (SSR) vs. PV & Wind Penetration Ratios',
    scene=dict(
        xaxis_title='PV Penetration Ratio (Installed PV Capacity / Average Demand)',
        yaxis_title='Wind Penetration Ratio (Installed Wind Capacity / Average Demand)',
        zaxis_title='Self-Sufficiency Rate (SSR) (%)'
    ),
    template='plotly_white',
    width=900,
    height=600,
    margin=dict(l=60, r=60, t=80, b=80),
    font=dict(size=10)
)

fig.show()
